# LLMChains & OutputParsers


Instead of just using models you can combine a model and a prompt. This can be done with the LLMChain class. We will additional, more complex chains in this notebook

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment? Sentiment is required.
subject: What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.
price: How much did the customer pay? Use 'None' if no price was provided.

Format the output as JSON with the following keys:
sentiment
subject
price

text: {input}
"""

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI()

prompt_template = ChatPromptTemplate.from_template(template=TEMPLATE)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.")

{'input': 'I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.',
 'text': '{\n    "sentiment": "neutral",\n    "subject": "food",\n    "price": "None"\n}'}